___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests

r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json')
if r.status_code == 200:
    print(True)
else:
    print(False)

True


2. Отримати курс долара за останній рік.

In [1]:
from datetime import datetime, timedelta
from statistics import mean

import requests

BASE = [] 

def con_to_api(date):
    param = {'date' : date.strftime('%Y%m%d'), 'valcode' : 'USD'}
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json', params=param)
    if r.status_code == 200:
        return r.json()[0]['rate']
    return None


def union_info(qty):
    
    date = datetime.today()
    
    for i in range(qty):
        date = date - timedelta(days=1)
        rate = con_to_api(date)
        BASE.append(rate)


if __name__ == '__main__':
    qty = 365
    union_info(qty)
    print(f'Курс USD за останній рік становить {mean(BASE)}')

Курс USD за останній рік становить 27.722214246575344


### або такий розвязок (залежно від того, як трактувати завдання)

In [2]:
import datetime

today = datetime.date.today()
rates = {}

for i in range(0, 366):
    day = today - datetime.timedelta(i)
    params    = {'date' : day.strftime('%Y%m%d')}
    
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&json', params).json()
    
    rates[day] = r[0]['rate']
    
print(rates)

{datetime.date(2021, 4, 21): 28.0096, datetime.date(2021, 4, 20): 28.0087, datetime.date(2021, 4, 19): 27.9783, datetime.date(2021, 4, 18): 27.9592, datetime.date(2021, 4, 17): 27.9592, datetime.date(2021, 4, 16): 27.9592, datetime.date(2021, 4, 15): 27.9765, datetime.date(2021, 4, 14): 28.0156, datetime.date(2021, 4, 13): 27.9335, datetime.date(2021, 4, 12): 27.9094, datetime.date(2021, 4, 11): 27.9768, datetime.date(2021, 4, 10): 27.9768, datetime.date(2021, 4, 9): 27.9768, datetime.date(2021, 4, 8): 27.8923, datetime.date(2021, 4, 7): 27.8384, datetime.date(2021, 4, 6): 27.939, datetime.date(2021, 4, 5): 27.9555, datetime.date(2021, 4, 4): 27.8324, datetime.date(2021, 4, 3): 27.8324, datetime.date(2021, 4, 2): 27.8324, datetime.date(2021, 4, 1): 27.8226, datetime.date(2021, 3, 31): 27.8852, datetime.date(2021, 3, 30): 27.9694, datetime.date(2021, 3, 29): 27.9679, datetime.date(2021, 3, 28): 27.9698, datetime.date(2021, 3, 27): 27.9698, datetime.date(2021, 3, 26): 27.9698, datetime.d

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [3]:
from datetime import date, timedelta, datetime
from statistics import mean, pstdev

import requests

BASE = [] 

def con_to_api(date):
    param = {'date' : date.strftime('%Y%m%d'), 'valcode' : 'USD'}
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json', params=param)
    if r.status_code == 200:
        return r.json()[0]['rate']
    return None

def period(month, year):
    day = timedelta(days=1)
    date1 = date(year, month, 1)
    dates = []
    d = date1
    while d.month == month:
        dates.append(d)
        d += day
    return dates

def union_info(m, y):
    
    dates = period(m, y)
    
    for d in dates:
        rate = con_to_api(d)
        BASE.append(rate)


if __name__ == '__main__':
    month = int(input('Введіть місяць для аналітики: '))
    year = int(input('Введіть рік для аналітики: '))
    union_info(month, year)
    month_name = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()[month-1]
    s1 = f"Середнє значення USD за {month_name} {year} року становить = {mean(BASE)}"
    s2 = f"Відхилення курсу USD за {month_name} {year} року становить = {pstdev(BASE)}"
    print(s1)
    print(s2)
    result = [s1, s2]

Введіть місяць для аналітики: 2
Введіть рік для аналітики: 2021
Середнє значення USD за Feb 2021 року становить = 27.884660714285715
Відхилення курсу USD за Feb 2021 року становить = 0.10439519125243588


4. Дану інформацію записати у файл за допомогою pickle.

In [5]:
import pickle

Lab_4 = open("Lab_4.pickle", "wb")
pickle.dump(result, Lab_4)
Lab_4.close()


In [6]:
import pickle

pickle_off = open("Lab_4.pickle","rb")
Lab4 = pickle.load(pickle_off)
pickle_off.close()
print(Lab4)

['Середнє значення USD за Feb 2021 року становить = 27.884660714285715', 'Відхилення курсу USD за Feb 2021 року становить = 0.10439519125243588']


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [216]:
import sqlite3

conn = sqlite3.connect('my_data.sqlite3')
conn.close()

conn = sqlite3.connect('my_data.sqlite3')
curs = conn.cursor()

script = """CREATE TABLE product ( 
                product_code TEXT NOT NULL PRIMARY KEY,
                product_name TEXT NOT NULL,
                price REAL NOT NULL,
                weight TEXT NOT NULL
                ) 
                """

curs.executescript(script)
conn.close()

In [217]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

import requests
from bs4 import BeautifulSoup

BASE = []

SITE = 'https://smallpacking.agrosem.ua/shop/'

r = requests.get(SITE, headers = HEADERS)

if r.status_code == 200:
    soup = BeautifulSoup(r.text, 'lxml')
    
    for el in soup.find('div', class_ = "product-category-wrapper").find_all('div', class_ = "product-from-category-container"):
        product_name = el.find('span', class_ = "pruduct-name-bottom").text
        unformatted_price = el.find('span', class_ = "regular-price").text.split('грн')
        price = int(unformatted_price[0].replace(' ',''))
        link = el.find('a', class_ = "more-detail").get('href')
        
        r_2 = requests.get(link, headers = HEADERS)
        
        if r_2.status_code == 200:
            soup_2 = BeautifulSoup(r_2.text, 'lxml')
            
            for s in soup_2.find_all('div', class_= "product-container"):
                product_code = s.find('span', class_= "sku").text.split(' ')[1].replace('SKU: ','')
                
            for w in soup_2.find_all('tr', class_ = "woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_fasovka-dobriv"):
                weight = w.find('td', class_ = "woocommerce-product-attributes-item__value").text
                
            BASE = [ (product_code, product_name, price, weight) ]
            
            for product_info in BASE:
                conn = sqlite3.connect('my_data.sqlite3')
                curs = conn.cursor()

                s = "INSERT INTO product VALUES (?, ?, ?, ?)"
                try:
                    curs.execute(s, product_info)
                    conn.commit()
                except Exception as e:
                    print(e)

                conn.close()

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [220]:
import sqlite3

def data_base():
    con = sqlite3.connect('tysql.sqlite')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master")
    info = cur.fetchall()
    print(info)
    con.close()
    
data_base()

[('Customers',), ('sqlite_autoindex_Customers_1',), ('OrderItems',), ('sqlite_autoindex_OrderItems_1',), ('Orders',), ('sqlite_autoindex_Orders_1',), ('Products',), ('sqlite_autoindex_Products_1',), ('Vendors',), ('sqlite_autoindex_Vendors_1',)]


2. Виведіть перелік всіх таблиць.

In [202]:
import sqlite3

def tables():
    con = sqlite3.connect('tysql.sqlite')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    info = cur.fetchall()
    print(info)
    con.close()
    
tables()

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [94]:
import sqlite3

def cust_id():
    cust_id_list = []
    
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    s = "SELECT cust_id FROM Customers"
    for el in curs.execute(s).fetchall():
        cust_id_list.append(el[0])
    conn.close()
    
    return cust_id_list
    
cust_id()

['1000000001', '1000000002', '1000000003', '1000000004', '1000000005']

4. Всю таблицю Customers table

In [100]:
import sqlite3

def data_base():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT * FROM Customers")

    print(curs.fetchall())
    conn.close()
    
data_base()

[('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com'), ('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None), ('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com'), ('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com'), ('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)]


5. Список клієнтів (cust_names) відсортованих від Z до A

In [108]:
import sqlite3

def cust_name():
    cust_name_list = []
    
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    l = "SELECT cust_name FROM Customers ORDER BY cust_name DESC"
    for el in curs.execute(l).fetchall():
        cust_name_list.append(el[0])
    conn.close()
    
    return cust_name_list

cust_name()

['Village Toys', 'The Toy Store', 'Kids Place', 'Fun4All', 'Fun4All']

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [112]:
import sqlite3

def orders():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT * FROM Orders ORDER BY cust_id, order_date")

    print(curs.fetchall())
    conn.close()

orders()

[(20009, '2012-02-08', '1000000001'), (20005, '2012-05-01', '1000000001'), (20006, '2012-01-12', '1000000003'), (20007, '2012-01-30', '1000000004'), (20008, '2012-02-03', '1000000005')]


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [121]:
import sqlite3

def items():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT quantity, item_price FROM OrderItems ORDER BY quantity DESC, item_price DESC")

    print(curs.fetchall())
    conn.close()

items()

[(250, 2.49), (250, 2.49), (250, 2.49), (100, 10.99), (100, 5.49), (100, 2.99), (100, 2.99), (100, 2.99), (50, 11.49), (50, 4.49), (20, 5.99), (10, 11.99), (10, 8.99), (10, 3.49), (10, 3.49), (10, 3.49), (5, 11.99), (5, 4.99)]


8. Товар (з таблиці Products), ціна якого становить 9.49

In [122]:
import sqlite3

def prod():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT prod_id, prod_name FROM Products WHERE prod_price = 9.49")

    print(curs.fetchall())
    conn.close()

prod()

[('RYL01', 'King doll'), ('RYL02', 'Queen doll')]


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [125]:
import sqlite3

def products():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT prod_name, prod_price FROM Products WHERE prod_price BETWEEN 3 AND 6 ORDER BY prod_price")

    print(curs.fetchall())
    conn.close()

products()

[('Fish bean bag toy', 3.49), ('Bird bean bag toy', 3.49), ('Rabbit bean bag toy', 3.49), ('Raggedy Ann', 4.99), ('8 inch teddy bear', 5.99)]


10. Кількість товару, що було продано

In [130]:
import sqlite3

def sales():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT prod_id, SUM(quantity) FROM OrderItems GROUP BY prod_id")

    print(curs.fetchall())
    conn.close()

sales()

[('BNBG01', 360), ('BNBG02', 360), ('BNBG03', 360), ('BR01', 120), ('BR02', 10), ('BR03', 165), ('RGAN01', 55)]


11. Кількість найменувань товару, ціна якого більша за 4

In [133]:
import sqlite3

def count():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    s = "SELECT COUNT(DISTINCT prod_name) FROM Products WHERE prod_price > 4"
    for el in curs.execute(s).fetchall():
        print(el[0])
    conn.close()

count()

6


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [134]:
import sqlite3

def top():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT prod_id, prod_name FROM Products ORDER BY prod_price DESC LIMIT 3")

    print(curs.fetchall())
    conn.close()

top()

[('BR03', '18 inch teddy bear'), ('RYL01', 'King doll'), ('RYL02', 'Queen doll')]


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [138]:
import sqlite3

def order():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT cust_id, cust_name, COUNT(order_num) FROM Orders INNER JOIN Customers USING (cust_id) GROUP BY cust_id")

    print(curs.fetchall())
    conn.close()

order()

[('1000000001', 'Village Toys', 2), ('1000000003', 'Fun4All', 1), ('1000000004', 'Fun4All', 1), ('1000000005', 'The Toy Store', 1)]


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [145]:
import sqlite3

def order_():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT cust_id, cust_name, order_num FROM Orders INNER JOIN Customers USING (cust_id) GROUP BY order_num")

    print(curs.fetchall())
    conn.close()

order_()

[('1000000001', 'Village Toys', 20005), ('1000000003', 'Fun4All', 20006), ('1000000004', 'Fun4All', 20007), ('1000000005', 'The Toy Store', 20008), ('1000000001', 'Village Toys', 20009)]


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [152]:
import sqlite3

def products_():
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    curs.execute("SELECT prod_name, prod_price FROM Products WHERE prod_price < 5 UNION SELECT prod_name, prod_price FROM Products WHERE prod_price >= 5 ORDER BY prod_price")

    print(curs.fetchall())
    conn.close()

products_()

[('Bird bean bag toy', 3.49), ('Fish bean bag toy', 3.49), ('Rabbit bean bag toy', 3.49), ('Raggedy Ann', 4.99), ('8 inch teddy bear', 5.99), ('12 inch teddy bear', 8.99), ('King doll', 9.49), ('Queen doll', 9.49), ('18 inch teddy bear', 11.99)]


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [56]:
## Зробила скріни із виконаними завданнями¶